### Generate test cases 10 times using different strategy

In [2]:
import subprocess
import os

encoding = 'A64'
arch = 'AArch32'
strategy = 'random-symbols'
if arch == 'AArch32':
    xml_file = '../../test-generator/mra_tools/v8.6/ISA_AArch32_xml_v86A-2019-12'
else:
    xml_file =  '../../test-generator/mra_tools/v8.6/ISA_A64_xml_v86A-2019-12'

# run the test case generator script
script_to_run = '../../test-generator/genInsts.py'
script_arguments = [
    '--altslicesyntax', 
    '--demangle', 
    '--verbose', 
    '-o', '../../test-generator/mra_tools/arch/arch', 
    xml_file, 
    '--encoding', encoding, 
    '--arch', arch, 
    '--strategy', strategy
]
try:
    if not os.path.exists(f"{strategy}/{encoding}"):
       os.makedirs(f"{strategy}/{encoding}")
    # Run the script using subprocess
    for i in range(10):
        subprocess.run(['python3', script_to_run] + script_arguments, check=True)
        subprocess.run(['mv', f'{encoding}.txt', f'{strategy}/{encoding}/{encoding}_{i}.txt'], check=True)
        subprocess.run(['mv', f'{encoding}_constraints.json', f'{strategy}/{encoding}/{encoding}_constraints_{i}.json'], check=True)

except subprocess.CalledProcessError as e:
    print(f"Error running {script_to_run}: {e}")


In [1]:
import json
overall = {
    'symbolic': {
        "number": 0,
        "cover_encoding": 0,
        "cover_instruction": 0,
        "cover_constraint": 0
    },
    'random': {
        "number": 0,
        "cover_encoding": 0,
        "cover_instruction": 0,
        "cover_constraint": 0
    },
    'random_symbols': {
        "number": 0,
        "cover_encoding": 0,
        "cover_instruction": 0,
        "cover_constraint": 0
    }
}

def calculate_statistics(instspath, encoding):
    results = {
        "number": [],
        "cover_encoding": [],
        "cover_instruction": [],
        "cover_constraint": []
    }
    for i in range(10):
        valid_file = f"{instspath}/{encoding}_{i}.txt" 
        with open(valid_file,"r") as f:
            lines = f.readlines()
            results["number"].append(len(lines))
        
        valid_constraint_file = f"{instspath}/{encoding}_constraints_{i}.json"
        tmp = set()
        with open(valid_constraint_file,"r") as f:
            constraint_file_data = json.load(f)
            results["cover_encoding"].append(len(constraint_file_data.keys()))
            results["cover_constraint"].append(str(constraint_file_data).count("True"))
            for k in constraint_file_data:
                if encoding == 'A64':
                    if '#' in k:
                        tmp.add(k.split("#")[1])
                    else:
                        tmp.add(k)
                elif '#' in k:
                    tmp.add(k.split("#")[1])
                else:
                    tmp.add(k.split("/")[-1])
        results["cover_instruction"].append(len(tmp))
    return results

def calculate_average(random, random_symbols, symbolic):
    mapping = {
        'number': 'Instruction Stream',
        'cover_encoding': 'Instruction Encoding',
        'cover_instruction': 'Instruction',
        'cover_constraint': 'Constraint'
    }
    for item in ['number', 'cover_encoding', 'cover_instruction', 'cover_constraint']:
        print(f"============={mapping[item]}================")
        overall['symbolic'][item] += sum(symbolic[item])/len(symbolic[item])
        overall['random'][item] += sum(random[item])/len(random[item])
        overall['random_symbols'][item] += sum(random_symbols[item])/len(random_symbols[item])
        print(f"symbolic       = {sum(symbolic[item])/len(symbolic[item])}")
        print(f"random         = {sum(random[item])/len(random[item])}")
        print(f"random_symbols = {sum(random_symbols[item])/len(random_symbols[item])}")
    

for encoding in ['A64', 'A32', 'T32', 'T16']:
    print(f"=============================={encoding}===========================")
    random = calculate_statistics(f"random/{encoding}", encoding)
    random_symbols = calculate_statistics(f"random-symbols/{encoding}", encoding)
    symbolic = calculate_statistics(f"symbolic/{encoding}", encoding)
    calculate_average(random, random_symbols, symbolic)
    print(f"=============================={encoding}==========================\n")

print(f"==============================Overall===========================")
print(f"symbolic       = {overall['symbolic']}")
print(f"random         = {overall['random']}")
print(f"random_symbols = {overall['random_symbols']}")


==============================A64===========================
=============Instruction Stream================
symbolic       = 1041355.4
random         = 421645.4
random_symbols = 573977.2
=============Instruction Encoding================
symbolic       = 837.0
random         = 265.6
random_symbols = 839.0
=============Instruction================
symbolic       = 579.0
random         = 265.6
random_symbols = 581.0
=============Constraint================
symbolic       = 3429.8
random         = 934.8
random_symbols = 3060.9
==============================A64==========================

==============================A32===========================
=============Instruction Stream================
symbolic       = 949704.6
random         = 582849.7
random_symbols = 641995.6
=============Instruction Encoding================
symbolic       = 550.0
random         = 414.6
random_symbols = 550.0
=============Instruction================
symbolic       = 481.0
random         = 7.9
random_symbols = 481

In [6]:
from scipy.stats import mannwhitneyu
from cliffs_delta import cliffs_delta

def element_wise_sum(d1, d2):
    new_dict = {}
    for key in d1.keys():
        new_dict[key] = [x + y for x, y in zip(d1[key], d2[key])]
    
    return new_dict

def utest(a, b):
    # Perform Mann-Whitney U test
    statistic, p_value= mannwhitneyu(a['cover_encoding'], b['cover_encoding'])
    result =  cliffs_delta(a['cover_encoding'], b['cover_encoding'])
    print(f"instruction encoding: {statistic}, {p_value}")
    print(f"instruction encoding: {result}")
    statistic, p_value = mannwhitneyu(a['cover_instruction'], b['cover_instruction'])
    result =  cliffs_delta(a['cover_instruction'], b['cover_instruction'])
    print(f"instruction: {statistic}, {p_value}")
    print(f"instruction: {result}")
    statistic, p_value = mannwhitneyu(a['cover_constraint'], b['cover_constraint'])
    result =  cliffs_delta(a['cover_constraint'], b['cover_constraint'])
    print(f"constraint: {statistic}, {p_value}")
    print(f"constraint: {result}")

encoding = 'A64'
random = calculate_statistics(f"random/{encoding}", encoding)
random_symbols = calculate_statistics(f"random-symbols/{encoding}", encoding)
symbolic = calculate_statistics(f"symbolic/{encoding}", encoding)
# print(random)

for encoding in ['A32', 'T32', 'T16']:
    print(f"========{encoding}===========")
    random_temp = calculate_statistics(f"random/{encoding}", encoding)
    random_symbols_temp = calculate_statistics(f"random-symbols/{encoding}", encoding)
    symbolic_temp = calculate_statistics(f"symbolic/{encoding}", encoding)
    random = element_wise_sum(random, random_temp)
    random_symbols = element_wise_sum(random_symbols, random_symbols_temp)
    symbolic = element_wise_sum(symbolic, symbolic_temp)
    # print(random)
    # print(random_symbols)
    # print(symbolic)

utest(symbolic, random)
print("="*16)
utest(symbolic, random_symbols)

========A32===========
========T32===========
========T16===========
instruction encoding: 100.0, 0.00016399116127605647
instruction encoding: (1.0, 'large')
instruction: 100.0, 0.00014417610153880437
instruction: (1.0, 'large')
constraint: 100.0, 0.00014589964277689183
constraint: (1.0, 'large')
instruction encoding: 0.0, 5.5941681768311536e-05
instruction encoding: (-1.0, 'large')
instruction: 0.0, 4.7682291610117017e-05
instruction: (-1.0, 'large')
constraint: 100.0, 0.00014589964277689183
constraint: (1.0, 'large')


In [7]:
from scipy.stats import mannwhitneyu
from cliffs_delta import cliffs_delta



def utest(a, b):
    # Perform Mann-Whitney U test
    statistic, p_value= mannwhitneyu(a['cover_encoding'], b['cover_encoding'])
    result =  cliffs_delta(a['cover_encoding'], b['cover_encoding'])
    print(f"instruction encoding: {statistic}, {p_value}")
    print(f"instruction encoding: {result}")
    statistic, p_value = mannwhitneyu(a['cover_instruction'], b['cover_instruction'])
    result =  cliffs_delta(a['cover_instruction'], b['cover_instruction'])
    print(f"instruction: {statistic}, {p_value}")
    print(f"instruction: {result}")
    statistic, p_value = mannwhitneyu(a['cover_constraint'], b['cover_constraint'])
    result =  cliffs_delta(a['cover_constraint'], b['cover_constraint'])
    print(f"constraint: {statistic}, {p_value}")
    print(f"constraint: {result}")


for encoding in ['A64', 'A32', 'T32', 'T16']:
    print(f"========{encoding}===========")
    random_temp = calculate_statistics(f"random/{encoding}", encoding)
    random_symbols_temp = calculate_statistics(f"random-symbols/{encoding}", encoding)
    symbolic_temp = calculate_statistics(f"symbolic/{encoding}", encoding)
    
    print(random)
    print(random_symbols)
    print(symbolic)
    utest(symbolic, random)
    print("="*16)
    utest(symbolic, random_symbols)

========A64===========
{'number': [1075285, 1035510, 1035218, 1034993, 1036878, 1036787, 1035648, 1036446, 1036410, 1035730], 'cover_encoding': [1085, 1087, 1091, 1084, 1089, 1092, 1088, 1085, 1093, 1095], 'cover_instruction': [289, 284, 282, 280, 287, 286, 280, 281, 287, 292], 'cover_constraint': [7926, 7876, 7937, 7935, 7893, 7976, 7979, 7939, 7959, 7962]}
{'number': [1692410, 1696858, 1694415, 1697284, 1699049, 1688311, 1691975, 1690422, 1697624, 1701713], 'cover_encoding': [1998, 1998, 1998, 1998, 1998, 1998, 1998, 1998, 1998, 1998], 'cover_instruction': [1581, 1581, 1581, 1581, 1581, 1581, 1581, 1581, 1581, 1581], 'cover_constraint': [12771, 12767, 12791, 12778, 12793, 12753, 12784, 12773, 12790, 12783]}
{'number': [2885423, 2840984, 2849953, 2877776, 2845922, 2863218, 2844942, 2902523, 2898515, 2849841], 'cover_encoding': [1994, 1995, 1995, 1994, 1995, 1994, 1993, 1994, 1993, 1993], 'cover_instruction': [1577, 1578, 1578, 1577, 1578, 1577, 1577, 1577, 1576, 1577], 'cover_constrai